In [1]:
# Basic Necessities
import pandas as pd
import numpy as np

# Tf-IDf Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
train_data = pd.read_csv('data/train_added_features.csv')
train_data.drop('Unnamed: 0',axis=1,inplace=True)
test_data = pd.read_csv('data/test_added_features.csv')
test_data.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
y_labels= train_data['sentiment']
sentiment = list(map(lambda x: 2 if x=="positive" else 1 if x=="neutral" else 0, y_labels))
train_data.drop('sentiment',axis=1,inplace=True)
train_data['sentiment']=sentiment

In [4]:
X = train_data['clean_text']
y = train_data['sentiment']

cv=TfidfVectorizer(ngram_range=(1,2), max_features=40000,min_df=2)
X = cv.fit_transform(X)
print(X.shape)

(21465, 29285)


In [5]:
train_data['tokenized'] = list(X.toarray())

# Feature Selection

In [6]:
# The final feature selection in our code is :
        # Exclaimation marks, positive emojis, negative emojis, positive words and negative words (5 features)

In [7]:
def unify(arr1,arr2,arr3,arr4,arr5,arr6):
    union = list()
    for i,v in enumerate(arr1):
        # features is a numpy array that will contain all the features of the i-th tweet
        features = np.append(arr1[i],arr2[i])
        features = np.append(features,arr3[i])
        features = np.append(features,arr4[i])
        features = np.append(features,arr5[i])
        features = np.append(features,arr6[i])
        #features = np.append(features,arr7[i])
        #features = np.append(features,arr8[i])
        #features = np.append(features,arr9[i])
        #features = np.append(features,arr10[i])
        #features = np.append(features,arr11[i])

        union.append(features)
    return np.asarray(union)

In [8]:
tok = train_data['tokenized'].tolist()
exc = train_data['number_of_exclamation'].tolist()
#que = train_data['number_of_questionmark'].tolist()
#tag = train_data['number_of_hashtag'].tolist()
#men = train_data['number_of_mention'].tolist(
pos_emo = train_data['number_of_positive_emo'].tolist()
neg_emo = train_data['number_of_negative_emo'].tolist()
pos_word = train_data['number_of_positive_words'].tolist()
neg_word = train_data['number_of_negative_words'].tolist()

X = unify(tok,exc,pos_emo,neg_emo,pos_word,neg_word)


In [9]:
y = train_data['sentiment']

In [10]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(max_iter=1000)
LR.fit(X,y)

LogisticRegression(max_iter=1000)

In [11]:
test_tweet = test_data['clean_text'].tolist()
test = cv.transform(test_tweet)
test_data['tokenized'] = list(test.toarray())

In [12]:
tok = test_data['tokenized'].tolist()
exc = test_data['number_of_exclamation'].tolist()
#que = test_data['number_of_questionmark'].tolist()
#tag = test_data['number_of_hashtag'].tolist()
#men = test_data['number_of_mention'].tolist()
#url = test_data['number_of_url'].tolist()
#quo = test_data['number_of_quotes'].tolist()
pos_emo = test_data['number_of_positive_emo'].tolist()
neg_emo = test_data['number_of_negative_emo'].tolist()
pos_word = test_data['number_of_positive_words'].tolist()
neg_word = test_data['number_of_negative_words'].tolist()

X_test = unify(tok,exc,pos_emo,neg_emo,pos_word,neg_word)


In [13]:
y_pred = LR.predict(X_test)
sentiment = list(map(lambda x: 'positive' if x==2 else 'neutral' if x==1 else 'negative', y_pred))
tweet_id = list(test_data['tweet_id'])
d = {'tweet_id': tweet_id, 'sentiment': sentiment}
submission = pd.DataFrame(data=d)
submission.set_index('tweet_id',inplace=True)
submission.head()
submission.to_csv('submission1.csv')

In [14]:
submission['sentiment'].value_counts()

neutral     2688
positive    2267
negative     443
Name: sentiment, dtype: int64